# Importações

In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

2022-03-22 15:06:51.183531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 15:06:51.183598: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Aquisição de dados

In [2]:
from RainPredictionMachine.data import CleanDataRpm
clean_data = CleanDataRpm()
df = clean_data.clean_data(1)
df.shape

(8784, 23)

# Processamento

In [3]:
from RainPredictionMachine.Trainer import pipe_creator
from sklearn import set_config
full_pipe, X_train, y_train  = pipe_creator(df)
set_config(display="diagram")
full_pipe

/home/victor/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('column_stransformer',
                 ColumnTransformer(transformers=[('scaling ',
                                                  Pipeline(steps=[('stdscaler',
                                                                   StandardScaler())]),
                                                  ['Pres', 'Pres_max',
                                                   'Pres_min', 'Radiacao',
                                                   'Temp', 'Temp_orvalho',
                                                   'Temp_max', 'Temp_min',
                                                   'Temp_orvalho_max',
                                                   'Temp_orvalho_min',
                                                   'Umid_max', 'Umid_min',
                                                   'Umid', 'Rajada_vento',
                                                   'Vel_vento'])]))])

In [4]:
X = pd.DataFrame(full_pipe.transform(X_train))

In [6]:
X.shape


(6148, 15)

expandindo dimensões

In [7]:
X_train_t1 = np.expand_dims(X, 2)
#X_test_t1 = np.expand_dims(X_test, 2)
X_train_t1

array([[[ 0.93561264],
        [ 0.87425622],
        [ 0.91392514],
        ...,
        [ 0.20770711],
        [-1.85252268],
        [-1.29311197]],

       [[-0.10775067],
        [-0.17683258],
        [-0.09692017],
        ...,
        [-1.27395271],
        [-0.5529953 ],
        [-0.81992555]],

       [[-0.78452687],
        [-0.74498869],
        [-0.71465898],
        ...,
        [-0.38495682],
        [-0.80451673],
        [-1.19847469]],

       ...,

       [[-0.5589348 ],
        [-0.46091064],
        [-0.49002669],
        ...,
        [-2.51854697],
        [ 0.07580826],
        [ 0.03181001]],

       [[-0.05135265],
        [-0.12001697],
        [-0.15307825],
        ...,
        [-0.85908796],
        [-0.93027744],
        [-0.91456283]],

       [[-1.48950208],
        [-1.56881505],
        [-1.44471393],
        ...,
        [ 0.74110465],
        [-0.63683578],
        [-0.34673913]]])

In [8]:
X_train_t1.shape

(6148, 15, 1)

# construção da RNN

## normalização

In [9]:

#normalizer = Normalization()
#normalizer.adapt(X_train_t1)
#normalizer(X_train_t1)[0]

norm = Normalization()

model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
model.fit(X_train_t1,y_train)


model.summary()

2022-03-22 15:07:55.065171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-22 15:07:55.065263: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-22 15:07:55.065338: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-9BMAD68): /proc/driver/nvidia/version does not exist
2022-03-22 15:07:55.065824: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


193/193 [==============================] - 12s 29ms/step - loss: 0.4910 - accuracy: 0.8941
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 15, 1)            3         
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 20)                1760      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 4)                 44        
                                                                 
Total params: 2,017
Trainable params: 2,014
Non-trainable params: 3
_________________________________________________________________


## camadas da RNN

In [36]:
#model = Sequential()
#model.add(normalizer)
#model.add(LSTM(units=15, activation='tanh'))
#model.add(Dense(10, activation='tanh'))
#model.add(Dense(4, activation='softmax'))


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

## compilando

In [ ]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    return model

model = compile_model(model)

# Ajuste da RNN

Fazendo um early stopping

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

Ajustando o modelo

In [ ]:
model.fit(X_train, y_train,
          epochs=100, 
          batch_size=32, 
          validation_split=0.2,
          callbacks=[es])

In [ ]:
df


In [ ]:
df['classe_chuva'].shift(-1)